In [2]:
import warnings
warnings.filterwarnings('ignore')

import torch
from torch import nn
from torchvision.models import resnet18
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
from skimage.io import imread
import numpy as np
import os
import csv
import xml.etree.ElementTree as ET
import cv2

In [3]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        resnet = resnet18(pretrained=True)
        resnet.avgpool = Identity()
        resnet.fc = Identity()

        self.downsampling = resnet

        self.cat1 = nn.Conv2d(64, 128, 1)
        self.cat2 = nn.Conv2d(128, 128, 1)
        self.cat3 = nn.Conv2d(256, 128, 1)
        
        self.upsampling_1 = nn.Sequential(nn.ReLU(),
                                          nn.ConvTranspose2d(512, 128, 3, 2, 1, 1)
                                         )

        self.upsampling_2 = nn.Sequential(nn.ReLU(),
                                          nn.BatchNorm2d(256),
                                          nn.ConvTranspose2d(256, 128, 3, 2, 1, 1)
                                         )

        self.upsampling_3 = nn.Sequential(nn.ReLU(),
                                          nn.BatchNorm2d(256),
                                          nn.ConvTranspose2d(256, 128, 3, 2, 1, 1)
                                         )

        self.upsampling_4 = nn.Sequential(nn.ReLU(),
                                         nn.BatchNorm2d(256),
                                         nn.ConvTranspose2d(256, 4, 3, 1, 1) 
                                        )

    def forward(self, x):
        x = self.downsampling.conv1(x)
        x = self.downsampling.bn1(x)
        x = self.downsampling.relu(x)
        x = self.downsampling.maxpool(x)

        down1 = self.downsampling.layer1(x)
        cat1 = self.cat1(down1)
        down2 = self.downsampling.layer2(down1)
        cat2 = self.cat2(down2)
        down3 = self.downsampling.layer3(down2)
        cat3 = self.cat3(down3)
        down4 = self.downsampling.layer4(down3)  

        up1 = self.upsampling_1(down4)
        up2 = self.upsampling_2(torch.cat((up1, cat3), 1))
        up3 = self.upsampling_3(torch.cat((up2, cat2), 1))
        up4 = self.upsampling_4(torch.cat((up3, cat1), 1))
        
        return up4       

### Convert xml to csv 

In [5]:
xml_folder_path = 'drive-download-20190824T073148Z-001/Gokce_Can/annotations/'
csv_path = 'drive-download-20190824T073148Z-001/Gokce_Can/labels.csv'
img_path = 'drive-download-20190824T073148Z-001/Gokce_Can/images/'

with open(csv_path, 'w') as csvfile:
    fieldnames = ['image_name', 'part_name', 'center_x', 'center_y']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    for xml_name in os.listdir(xml_folder_path):
        xml_path = xml_folder_path + xml_name
        tree = ET.parse(xml_path)
        root = tree.getroot()

        img_name = root.find('filename').text
        objects = root.findall('object')

        for part in objects:
            name = part.find('name').text
            object_bb = part.find('bndbox')
            xmin = int(object_bb.find('xmin').text)
            xmax = int(object_bb.find('xmax').text)
            ymin = int(object_bb.find('ymin').text)
            ymax = int(object_bb.find('ymax').text)

            center_x = (xmin+xmax) // 2
            center_y = (ymin+ymax) // 2

            writer.writerow({'image_name': img_name, 'part_name': name,
                             'center_x': center_x, 'center_y': center_y})            


In [6]:
class detection_dataset(Dataset):
    def __init__(self, img_size, img_folder_path, csv_path, transform=True):
        self.height = img_size[0]
        self.width = img_size[1]
        self.folder_path = img_folder_path
        self.img_files = os.listdir(img_folder_path)
        self.csv_path = csv_path
        self.transform = transform
        self.get_annotations()
        
    def __len__(self):
        return (len(self.img_files))
    
    def __getitem__(self, index):
        img_name = self.img_files[index]
        img = imread(self.folder_path + img_name)
        annotation = self.annotations[img_name]
        
        if self.transform:
            img, annotation = torch.from_numpy(img), torch.from_numpy(annotation)

        img = img.type(torch.FloatTensor)
        annotation = annotation.type(torch.FloatTensor)
        sample = {'image':img, 'annotation':annotation}

        return sample
          
    def get_annotations(self):
        self.annotations = defaultdict(lambda: np.zeros([4, self.height//4, self.width//4]))
        part_mapping = {'Head':0, 'Foot':1, 'Hand':2, 'Trunk':3}
        with open(self.csv_path) as csvfile:
            readCSV = csv.reader(csvfile)
            for row in readCSV:
                img_name = row[0]
                image_annotation = self.annotations[img_name]
                channel = part_mapping[row[1].capitalize()]
                x = int(row[3]) // 4
                y = int(row[2]) // 4
                image_annotation[channel, x-8:x+8, y-8:y+8] = makeGaussian(16,8)
                

def makeGaussian(size, fwhm = 3):
    ''' 
    Input: size: length of a side of the square
           fwhm: full-width-half-maximum (effective radius)
    Output: a square gaussian kernel
    Reference: https://stackoverflow.com/a/14525830
    '''

    x = np.arange(0, size, 1, float)
    y = x[:,np.newaxis]

    x0 = y0 = size // 2

    return np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / fwhm**2)

In [ ]:
# train only the decoder weights for 50 epochs 
csv_path = 'drive-download-20190824T073148Z-001/Gokce_Can/labels.csv'
img_path = 'drive-download-20190824T073148Z-001/Gokce_Can/images/'

dg = detection_dataset((1280,784), img_path, csv_path)
dataloader = DataLoader(dg, batch_size=20, shuffle=True)

criterion = nn.MSELoss()
model = Model()

trainable_parameters = [list(model.upsampling_1.parameters()) +\
                        list(model.upsampling_2.parameters()) +\
                        list(model.upsampling_3.parameters()) +\
                        list(model.upsampling_4.parameters())]
#optimizer = Adam(model.parameters())
optimizer = Adam(trainable_parameters)
model.cuda()
criterion.cuda()

epochs = 50

for epoch in range(epochs):
    for data in dataloader:
        img, bb = data['image'], data['annotation']
        img, bb = img.cuda(), bb.cuda()   
        pred = model(img)   
        loss = criterion(pred, bb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # calculate precision and false error percentage

### Train the entire model for 50 more epochs

In [ ]:
optimizer = Adam(model.parameters())

for epoch in range(epochs):
    for data in dataloader:
        img, bb = data['image'], data['annotation']
        img, bb = img.cuda(), bb.cuda()   
        pred = model(img)   
        loss = criterion(pred, bb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [1]:
def decode_network_output(out):
    head = out(..., 0) * 255
    foot = out(..., 1) * 255
    hands = out(..., 2) * 255
    trunk = out(..., 3) * 255
    
    blurred_head = cv2.GaussianBlur(head, (5, 5), 0)
    blurred_foot = cv2.GaussianBlur(foot, (5,5), 0)
    blurred_hands = cv2.GaussianBlur(hands, (5,5), 0)
    blurred_trunk = cv2.GaussianBlur(trunk, (5,5), 0)
    
    thresh_head = cv2.threshold(blurred_head, 60, 255, cv2.THRESH_BINARY)[1]
    thresh_foot = cv2.threshold(blurred_head, 60, 255, cv2.THRESH_BINARY)[1]
    thresh_hands = cv2.threshold(blurred_head, 60, 255, cv2.THRESH_BINARY)[1]
    thresh_trunk = cv2.threshold(blurred_head, 60, 255, cv2.THRESH_BINARY)[1]
    
    cnts_head = cv2.findContours(thresh_head.copy(), cv2.RETR_EXTERNAL,
           cv2.CHAIN_APPROX_SIMPLE)
    cnts_head = imutils.grab_contours(cnts_head)
    
    cnts_foot = cv2.findContours(thresh_foot.copy(), cv2.RETR_EXTERNAL,
           cv2.CHAIN_APPROX_SIMPLE)
    cnts_foot = imutils.grab_contours(cnts_foot)
    
    cnts_hands = cv2.findContours(thresh_hands.copy(), cv2.RETR_EXTERNAL,
           cv2.CHAIN_APPROX_SIMPLE)
    cnts_hands = imutils.grab_contours(cnts_hands)
    
    cnts_trunk = cv2.findContours(thresh_trunk.copy(), cv2.RETR_EXTERNAL,
           cv2.CHAIN_APPROX_SIMPLE)
    cnts_trunk = imutils.grab_contours(cnts_trunk)  
    
    heads = []
    foot = []
    hands = []
    trunk = []
    
    for c in cnts_head:
        M = cv2.moments(c)
        c_x = int(M["m10"] / M["m00"])
        c_y = int(M["m01"] / M["m00"])
        heads.append([c_x, c_y])
        
    for c in cnts_foot:
        M = cv2.moments(c)
        c_x = int(M["m10"] / M["m00"])
        c_y = int(M["m01"] / M["m00"])
        foot.append([c_x, x_y])
        
    for c in cnts_hands:
        M = cv2.moments(c)
        c_x = int(M["m10"] / M["m00"])
        c_y = int(M["m01"] / M["m00"])
        hands.append([c_x, x_y])
        
    for c in cnts_trunk:
        M = cv2.moments(c)
        c_x = int(M["m10"] / M["m00"])
        c_y = int(M["m01"] / M["m00"])        
        trunk.append([c_x, x_y])
        
    return [heads, foot, hands, trunk]